In [15]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# 데이터 로드 (헤더가 없음을 명시)
df = pd.read_csv('post_list.csv', encoding='cp949', header=None)

# 열 이름 지정
df.columns = ['Title', 'AuthorGrade']

# 텍스트 데이터와 레이블 분리
texts = df['Title'].values
labels = df['AuthorGrade'].values

# 텍스트 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 시퀀스 패딩
max_len = 100  # 시퀀스의 최대 길이
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# 학습 데이터와 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# AdaBoost 모델 생성
base_estimator = DecisionTreeClassifier(max_depth=1)
model = AdaBoostClassifier(estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

# 모델 학습
model.fit(X_train, y_train)

# 검증 데이터로 평가
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print('accuracy:', accuracy_score(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.45      0.51        67
           1       0.37      0.58      0.45        38
           2       0.77      0.82      0.79        44
           3       0.81      0.81      0.81        21
           4       1.00      0.75      0.86        28

    accuracy                           0.64       198
   macro avg       0.71      0.68      0.68       198
weighted avg       0.67      0.64      0.64       198

accuracy: 0.6363636363636364


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [16]:
# 등급 매핑 사전 정의
grade_mapping = {0: '진드기', 1: '닭둘기', 2: '왁무새', 3: '침팬치', 4: '느그자'}

# 예측 함수 정의
def predict_author_grade(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded_sequence)
    return prediction[0]

# 예측 예시
example_text = "0글 해야되니까 나가주세요 핸님들"
predicted_grade = predict_author_grade(example_text)
grade_text = grade_mapping[predicted_grade]
print(f'당신의 등급은 {grade_text}입니다.')

당신의 등급은 진드기입니다.
